In [11]:
import torch
from torch import nn
import torch.nn.functional as F
import math

In [12]:
#验证库的有效性
random_torch=torch.rand(4,4)
print(random_torch)

tensor([[0.2758, 0.0750, 0.6040, 0.0393],
        [0.9600, 0.4581, 0.0760, 0.8345],
        [0.0242, 0.3250, 0.5424, 0.5312],
        [0.8575, 0.7061, 0.4172, 0.4962]])


In [13]:
#将输入的词汇表索引转换为指定维度的embedding
class TokenEmbedding(nn.Embedding):
    def __init__(self,vocab_size,d_model):#词汇表的大小与embedding的维度
        super(TokenEmbedding,self).__init__(vocab_size,d_model,padding_idx=1)

- nn.Embedding 的参数是 (num_embeddings, embedding_dim, padding_idx=None)
- 当输入一个批次的句子（形状 [batch_size, seq_len]），TokenEmbedding 会把每个 token 的 id 转换成一个 d_model 维的向量。

In [17]:
#位置嵌入
class PositionalEmbedding(nn.Module):
    def __init__(self,d_model,max_len,device):
        super( PositionalEmbedding,self).__init__()
        self.encoding=torch.zeros(max_len,d_model,device=device)
        self.encoding.requires_grad =False
        
        #以上为位置编码 (Positional Encoding)，让模型知道句子中单词的顺序。
        
        
        pos=torch.arange(0,max_len,device=device) #表示序列中每个位置
        pos=pos.float().unsqueeze(dim=1) #变成列向量，方便后续广播计算
        _2i=torch.arange(0,d_model,step=2,device=device).float() #代表偶数位置的维度索引
        #设置位置编码 奇数位置偶数位置
        self.encoding[:,0::2]=torch.sin(pos/(10000**(_2i/d_model)))
        self.encoding[:,1::2]=torch.cos(pos/(10000**(_2i/d_model)))
        #前向传播的过程
    
    def forward(self,x):
        batch_size,seq_len=x.size()
        return self.encoding[:seq_len,:]
    

In [18]:
#两部分结合 以及dropout
class TransformerEmbedding(nn.Module):
    def __init__(self,vocab_size,d_model,max_len,drop_prob,device):
        super(TransformerEmbedding,self).__init__()
        self.tok_emb=TokenEmbedding(vocab_size,d_model)
        self.pos_emb=PositionalEmbedding(d_model,max_len,device)
        self.drop_out=nn.Dropout(p=drop_prob)
        
    def forward(self,x):
        tok_emb=self.tok_emb(x)
        pos_emb=self.pos_emb(x)
        return self.drop_out(tok_emb+pos_emb)

- tok_emb(x)：把 [batch_size, seq_len] 的 token id 转换为 [batch_size, seq_len, d_model] 的词向量。
- pos_emb(x)：生成 [seq_len, d_model] 的位置编码（可能需要扩展成 [batch_size, seq_len, d_model]）。
- tok_emb + pos_emb：把词向量和位置信息相加。
- Dropout：正则化，防止过拟合。
### 最终输出 [batch_size, seq_len, d_model]，作为 Transformer 的输入。

# 数据流动示意

In [19]:
if __name__ == "__main__":
    # 假设参数
    vocab_size = 10   # 词表大小
    d_model = 6       # embedding维度
    max_len = 20      # 位置编码支持的最大长度
    drop_prob = 0.1
    device = "cpu"

    # 模型
    emb = TransformerEmbedding(vocab_size, d_model, max_len, drop_prob, device)

    # 输入数据：batch_size=2，seq_len=4
    x = torch.tensor([
        [2, 5, 7, 1],   # 第一句话 (4个token)
        [3, 9, 4, 1]    # 第二句话
    ])

    print("输入 x 的形状:", x.shape)   # (2, 4)
    print("输入 x 的内容:\n", x)

    out = emb(x)

    print("\n输出 embedding 的形状:", out.shape)  # (2, 4, 6)
    print("输出 embedding 示例:\n", out)

输入 x 的形状: torch.Size([2, 4])
输入 x 的内容:
 tensor([[2, 5, 7, 1],
        [3, 9, 4, 1]])

输出 embedding 的形状: torch.Size([2, 4, 6])
输出 embedding 示例:
 tensor([[[ 0.1572,  2.4798, -0.7344,  2.2878, -0.9293,  0.9443],
         [ 0.0421,  0.0000,  1.7298,  0.0000, -1.3695, -0.1315],
         [ 0.6056,  0.1363, -1.4040,  0.7559,  2.5565, -0.2930],
         [ 0.1568, -1.1000,  0.1542,  1.1004,  0.0072,  1.1111]],

        [[ 0.0000,  0.7314, -0.6190,  0.0000, -1.1384,  0.2420],
         [-0.0000,  0.5796,  0.1451,  2.4555,  0.0000,  1.3581],
         [-0.4549, -0.0915,  0.3040,  2.1437,  0.7554,  0.5495],
         [ 0.1568, -1.1000,  0.1542,  1.1004,  0.0072,  1.1111]]],
       grad_fn=<MulBackward0>)


数据流向为:
1. 输入 token id：(batch_size, seq_len)
   例如 (2,4) → [[2,5,7,1],[3,9,4,1]]

2. TokenEmbedding：查词表 → (batch_size, seq_len, d_model)
   (2,4,6)

3. PositionalEmbedding：生成正弦/余弦位置编码 → (seq_len, d_model)
   广播成 (batch_size, seq_len, d_model)

4. 相加并 Dropout → (batch_size, seq_len, d_model)
   作为 Transformer 的输入。